## 1. Import libraries

In [11]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import sklearn
import requests
from bs4 import BeautifulSoup
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

True


## 2. Data collection

In [13]:
# define url
url = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'
response = requests.get(url)

# define the attributes for data
Name = []
Age = []
Hometown = []
Boarded = []
Destination = []
Lifeboat = []
Class = []
Survive = []

# check if the url is ok or not
if (response.status_code != 200):
    print('Error')
else:
    url_data = response.text
    soup = BeautifulSoup(url_data, 'html.parser')

    # find tags data
    passenger_tags = soup.find_all('table', {'class':'wikitable sortable'})
    clss = 0
    span_cnt = np.full(6, 0, dtype = int)
    for passenger_tag in passenger_tags:
        tr_tags = passenger_tag.find_all('tr')
        
        clss += 1
        check = np.full(6, True, dtype = bool)
        for tr_tag in tr_tags:
            td_tags = tr_tag.find_all('td')
            if not td_tags:
                continue
                
            for i, td_tag in enumerate(td_tags):
                if i >= 6:
                    continue

                row = 0
                if 'rowspan' in td_tag:
                    row = int(td_tag.get('rowspan'))
                    span_cnt[i] += row-1
                    check[i] = False
                else:
                    row = 1
                Class.append(clss)
                if check[i] == True:
                    if i == 0:
                        while row > 0:
                            Name.append(td_tag.text) if td_tag else Name.append(np.nan)
                            row -= 1
                    elif i == 1:
                        while row > 0:
                            Age.append(td_tag.text) if td_tag else Age.append(np.nan)
                            row -= 1
                    elif i == 3:
                        while row > 0:
                            Hometown.append(td_tag.text) if td_tag else Hometown.append(np.nan)
                            row -= 1
                    elif i == 4:
                        while row > 0:
                            Boarded.append(td_tag.text) if td_tag else Boarded.append(np.nan)
                            row -= 1
                    elif i == 5:
                        while row > 0:
                            Lifeboat.append(td_tag.text) if td_tag else Lifeboat.append(np.nan)
                            row -= 1
                else:
                    span_cnt[i] -= 1
                    if span_cnt[i] == 0:
                        check[i] = True
            

SyntaxError: expected ':' (2858434999.py, line 71)